In [ ]:
import googlemaps
import pandas as pd
import time

In [ ]:
# get your own API_KEY
# API_KEY = your_key

# 1. Define Client and fields

In [ ]:
gmaps = googlemaps.Client(key = API_KEY)

In [ ]:
# defining the fields that we need to search for each ID
my_fields = ['name', 'permanently_closed']

# 2. Getting the formatted address for each store

In [ ]:
priority_larry = pd.read_excel('Priority A_B_C_3.0_cleansed.xlsx', sheet_name='Larry_438 stores')

In [ ]:
priority_larry

In [ ]:
priority_larry['Country'] = 'Canada'
priority_larry['Formatted_address'] = priority_larry['Name'] + ',' + priority_larry['Street_Address'] + ',' + priority_larry['City__Town'] + ',' + \
priority_larry['Postal code'] + ',' + priority_larry['Country']


In [ ]:
priority_larry.head()

# 3. Getting the unique ID for each of the stores using geocode api

In [ ]:
priority_larry['id'] = 'na'
priority_larry_loa = priority_larry['Formatted_address']
for address in priority_larry_loa:
    try:
        priority_larry.loc[priority_larry['Formatted_address'] == address, ['id']] = gmaps.geocode(address)[0]['place_id']
        # set a buffer of 0.3 sec when running on google cloud
        time.sleep(0.3)
    except:
        break

In [ ]:
priority_larry.head()

# 4. Filtering the permanently_closed lists

In [ ]:
priority_larry['Permanently_closed'] = False

In [ ]:
priority_larry.head()

In [ ]:
priority_larry_id_loa = priority_larry['id']

In [ ]:
for id in priority_larry_id_loa[:]:
    try:
        gmaps.place(place_id = id, fields = my_fields)['result']['permanently_closed']
        priority_larry.loc[priority_larry['id'] == id, ['Permanently_closed']] = True
    except:
        continue

In [ ]:
priority_larry.to_csv("priority_larry_final.csv")